In [1]:
#imports

import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import array
from sklearn.model_selection import train_test_split
import tensorflow as tf
import torch.optim as optim
from sklearn.model_selection import GridSearchCV
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable
from sklearn import preprocessing
import tqdm
import time
import torch
import copy
from torch.utils.data import DataLoader, TensorDataset

In [2]:
#data_for_model

data = pd.read_csv('/content/drive/MyDrive/DataFrame.csv', index_col = 'Unnamed: 0' )
data = data.rename(columns = {'target' : 'chord_target', '99' : 'time_target'})
N = 9000
X = np.zeros((N, 99))
Y = np.zeros((N))
for i in range(N):
    row = data.iloc[i]
    x = row[:98]
    y = row[98:]
    for j in range(0, len(x)-1, 2):
        chord = x[j]
        duration = x[j+1]
        X[i][j] = chord
        X[i][j+1] = duration
    X[i][98] = row[98]
    Y[i] = y[1]

X = (X - X.mean()) / X.std()
X_train, X_validation, y_train, y_validation = train_test_split(X, Y, random_state=43, test_size=0.2)

In [3]:
#decode_and_encode

def decode_chord_data(chord_data):
    file = open("/content/drive/MyDrive/Decode.json")
    decoder = json.load(file)
    file.close()

    res = []
    for chord, duration in chord_data:
        res.append((decoder[str(chord)], duration))

    return res


def encode_chord_data(chord_data):

    file = open("/content/drive/MyDrive/Decode.json")
    decoder = json.load(file)
    file.close()

    encoder = dict()
    for key, value in decoder.items():
        encoder[value] = int(key)

    res = []
    for chord, duration in chord_data:
        res.append((encoder[chord], duration))

    return res

In [ ]:
device = 'cpu'
print(f"Using device: {device}")

class EarlyStopping():

  def __init__(self, patience=5, min_delta=1e-3, restore_best_weights=True):
    self.patience = patience
    self.min_delta = min_delta
    self.restore_best_weights = restore_best_weights
    self.best_model = None
    self.best_loss = None
    self.counter = 0
    self.status = ""


  def __call__(self, model, val_loss):
    if self.best_loss == None:
      self.best_loss = val_loss
      self.best_model = copy.deepcopy(model)
    elif self.best_loss - val_loss > self.min_delta:
      self.best_loss = val_loss
      self.counter = 0
      self.best_model.load_state_dict(model.state_dict())
    elif self.best_loss - val_loss < self.min_delta:
      self.counter += 1
      if self.counter >= self.patience:
        self.status = f"Stopped on {self.counter}"
        if self.restore_best_weights:
          model.load_state_dict(self.best_model.state_dict())
        return True


    self.status = f"{self.counter}/{self.patience}"
    return False

In [ ]:
#model

class Net(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

X_train = torch.Tensor(X_train).float()
y_train = torch.Tensor(y_train).float()

X_validation = torch.Tensor(X_validation).float().to(device)
y_validation = torch.Tensor(y_validation).float().to(device)

batch_size = 16

dataset_train = TensorDataset(X_train, y_train)
dataloader_train = DataLoader(dataset_train,\
  batch_size=batch_size)

dataset_test = TensorDataset(X_validation, y_validation)
dataloader_test = DataLoader(dataset_test,\
  batch_size=batch_size)

model = Net(X.shape[1], 4, 2, 1).to(device)

loss_fn = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

es = EarlyStopping()

epoch = 0
done = False
while epoch < 100 and not done:
  epoch += 1
  steps = list(enumerate(dataloader_train))
  pbar = tqdm.tqdm(steps)
  model.train()
  for i, (x_batch, y_batch) in pbar:
    y_batch_pred = model(x_batch.to(device)).flatten()
    loss = loss_fn(y_batch_pred, y_batch.to(device))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss, current = loss.item(), (i + 1)* len(x_batch)
    if i == len(steps)-1:
      model.eval()
      pred = model(X_validation).flatten()
      vloss = loss_fn(pred, y_validation)
      if es(model,vloss): done = True
      pbar.set_description(f"Epoch: {epoch}, tloss: {loss}, vloss: {vloss:>7f}, EStop:[{es.status}]")
    else:
      pbar.set_description(f"Epoch: {epoch}, tloss {loss:}")

In [ ]:
#predict

def predict(model, input):
    model.eval()
    with torch.no_grad():
        predictions = model(input)
    return predictions

In [ ]:
#convertation and filler

def convert_to_2D(arr):

  arr1 = np.zeros((1, len(arr)))
  for i in range(0, len(arr)):
      arr1[0][i] = arr[i]
  return arr1

def convert_to_1D(arr):

  arr1 = []
  for i in range(0, len(arr)):
      arr1.append(arr[i])
  return arr1

def filler(arr):

  for_model = []
  while(len(for_model) < 98):
      for_model += arr
  return for_model[-98:]

In [ ]:
#prediction

def predict_sequence(chord_data, n):
  chord_data = encode_chord_data(chord_data)
  sequence = [i for p in chord_data for i in p]
  result = list(chord_data)

  if len(sequence) > 98:
        sequence = sequence[-98:]
  elif len(sequence) < 98:
      sequence = filler(sequence)

  seq = sequence
  for i in range(n):

    predicted_chord = i//10 + i #random_chord
    seq.append(predicted_chord)
    seq = convert_to_2D(seq)
    seq = torch.Tensor(seq).float()
    predicted_duration = predict(model, seq)
    seq = seq.numpy()
    seq = np.insert(seq, 99, np.array([predicted_duration]))
    result.append((int(seq[98]), float(seq[99])))
    seq = seq[2:]
    seq = convert_to_1D(seq)

  return decode_chord_data(result)

a = predict_sequence((('Am', 1.2), ('Dm', 3.5), ('Am', 1.4), ('C', 1.3), ('F', 1.3)), n=3)